In [5]:
import numpy as np
import random
from random import choice
import math
from enum import Enum
import copy

In [6]:
P_FLAT = 0.25
P_HILL = 0.25 + P_FLAT
P_FOREST = 0.25 + P_HILL
P_CAVE = 0.25 + P_FOREST

In [7]:
FN_FLAT = 0.1
FN_HILL = 0.3
FN_FOREST = 0.7
FN_CAVE = 0.9
probability_dict = {"flat": 0.1, "hill":0.3, "forest":0.7, "cave":0.9}

In [8]:
class Rule(Enum):
    rule_1 = 1
    rule_2 = 2

In [9]:
def manhattan(x, y, a, b):
    return abs(x-a) + abs(y-b)

In [10]:
def buildmap(dim):
    terrainMap = np.random.random((dim, dim))
    flat = terrainMap < P_FLAT
    hill = terrainMap < P_HILL
    forest = terrainMap < P_FOREST
    cave = terrainMap < P_CAVE
    
    terrainMap[cave] = FN_CAVE
    terrainMap[forest] = FN_FOREST
    terrainMap[hill] = FN_HILL
    terrainMap[flat] = FN_FLAT
    
    return terrainMap

In [19]:
class target(object):
    def __init__(self, dim, terrainMap):
        self.x =random.randint(0, dim-1)
        self.y = random.randint(0, dim-1)
        self.prev = 0
        self.current = terrainMap[self.x][self.y]
    
    def move(self, dim, terrainMap):
        tmp_x, tmp_y = choice([(1,0),(-1,0),(0,1),(0,-1)])
        
        tmp_x += self.x
        tmp_y += self.y
        
        if tmp_x<0 or tmp_x>dim-1 or tmp_y<0 or tmp_y>dim-1:
            return self.move(dim, terrainMap)
        self.prev = terrainMap[self.x][self.y]
        
        self.x = tmp_x
        self.y = tmp_y
        
        self.current = terrainMap[self.x][self.y]
        report = set()
        report.add(self.prev)
        report.add(self.current)
        return report
    
    def getPosition(self):
        return (self.x, self.y)

In [27]:
def array_to_dict(type_FN, terrainMap):
    tmp= np.where(terrainMap == type_FN)
    tmp_dict = {}
    for i in range(len(tmp[0])):
        tmp_dict[(tmp[0][i], tmp[1][i])] = 1
    return tmp_dict

In [30]:
test_map = buildmap(10)
dict_test = {}
dict_test[FN_FLAT] = array_to_dict(FN_FLAT, test_map)
dict_test[FN_HILL] = array_to_dict(FN_HILL, test_map)
print(dict_test[FN_FLAT])

{(0, 2): 1, (0, 7): 1, (1, 1): 1, (1, 2): 1, (1, 3): 1, (1, 5): 1, (1, 6): 1, (1, 7): 1, (2, 7): 1, (3, 0): 1, (3, 2): 1, (3, 6): 1, (4, 9): 1, (5, 3): 1, (5, 6): 1, (5, 9): 1, (6, 2): 1, (6, 3): 1, (6, 4): 1, (6, 7): 1, (7, 8): 1, (8, 2): 1, (8, 5): 1, (8, 6): 1, (9, 0): 1, (9, 7): 1}


In [31]:
def get_junction(cell, type_cands, type_cells):
    x, y = cell
    neighbor_cand = [(1,0),(-1,0),(0,1),(0,-1)]
    for step in neighbor_cand:
        i, j = step
        if (x+i, y+j) in type_cands:
            type_cells[(x+i, y+j)] = 1

In [32]:
cell = (1,1)
type_cells = {}
get_junction(cell, dict_test[FN_FLAT], type_cells)
print(type_cells)

{(1, 2): 1}


In [ ]:
def initial_filter(type_1, type_2, terrainMap, types_dict, prob_terrain):
    if type_1 == type_2:
        types_dict[type_1] = array_to_dict(type_1, terrainMap)
        type_cells = {}
        for cell in types_dict[type_1]:
            get_junction(cell, types_dict[type_1], type_cells)
        types_dict[type_1] = type_cells
        prob = 1/sum(types_dict[type_1].value())
        for cell in types_dict[type_1]:
            prob_terrain[cell] = prob

In [ ]:
def filter_terrain(prev_type, next_type, types_dict, prob_terrain):
    

In [ ]:
class Agent_memory(object):
    def __init__(self, rule = Rule.rule_2):
        self.prob_terrain = {}
        self.types_dict = {}
        self.prev_types = set()
    def update_memory(self)

In [ ]:
class Agent(object):
    def __init__(self, rule = Rule.rule_2):
        self.reduced_terrain = {}
        self.types_dict = {}
        self.prev_types = set()
        
        self.x = random.randint(0, dim-1)
        self.y = random.randint(0, dim-1)
        self.rule = rule
        self.cost = 0
    
    def search_cell(self, x, y, target):
        targetx, targety = target
        p = random.random()
        tp = self.terrain[x][y]
        self.cost +=1
        self.x = x, self.y = y
        if self.x==targetx and self.y ==targety:
            if p > tp:
                print("Catch you!!")
                return True
            else:
                return False
            
    def rule(self, terrainMap = []):
        if self.rule == Rule.rule_1: return self.rule1_dist()
        else: return self.rule2_dist(terrainMap)
    
    def rule1_dist(self):
        max_prio = 0
        x =0, y=0
        for pair in self.reduced_terrain:
            i, j =pair
            if self.reduced_terrain[pair]/manhattan(i, j, self.x, self.y) > max_prio:
                max_prio = self.reduced_terrain[pair]/manhattan(i, j, self.x, self.y) 
                x = i, y = j
        self.cost += manhattan(x, y, self.x, self.y)
        return x, y
    
    def rule2_dist(self, terrainMap):
        max_prio = 0
        x = 0, y= 0
        
        if not terrainMap: return x, y
        
        for pair in self.reduced_terrain:
            i, j = pair
            p = self.reduced_terrain[pair]
            tmp = (1 - terrainMap[i][j]) * p/manhattan(i, j, self.x, self.y)
            if tmp > max_prio:
                max_prio = tmp
                x = i, y = j
        self.cost += manhattan(x, y, self.x, self.y)
        return x, y
    
    def analysis_report(self, report, terrainMap):
        if not self.prev_types:
            prev_types = copy.deepcopy(report)
            if len(report) ==1:
                type_FN = report.pop()
                self.types_dict[type_FN] = array_to_dict(type_FN, terrainMap)
                self.terrain_filter_one(type_FN, type_FN)
                return self.rule()
            else:
                type_FN_1 = report.pop(), type_FN_2 = report.pop()
                self.array_to_dict(type_FN_1)
                self.array_to_dict(type_FN_2)
                self.terrain_filter_two(type_FN_1, type_FN_2)
                return self.rule()
        
        if report == self.prev_types:
            return self.rule()
        
        if len(report) == 1:
            remove_terrain = (prev_types - report).pop()
            self.types_dict.pop(remove_terrain)
            prev_types = report
            type_FN = report.pop()
            self.terrain_filter_one(type_FN, type_FN)
            return self.rule()
        
        if len(report) == 2:
            remove_terrain = (prev_types - report).pop()
            self.types_dict.pop(remove_terrain)
            type_FN = (report - prev_types).pop()
            type_prev_FN = (report & prev_types).pop()
            self.terrain_filter_one(type_prev_FN, type_FN)
            return self.rule()
            
    
    def update_state(self, x, y, terrainMap):
        falseNeg = terrainMap[x][y]
        
                